## Prerequisites

#### I set up a virtual environment using Miniconda3 to run this.

```bash
conda create -n LAMMPS python=3.12.10
conda activate LAMMPS
conda install mamba     # The dependency solver with mamba is much better for this
mamba install -c conda-forge lammps
mamba install -c conda-forge ase  
mamba install -c conda-forge py3Dmol 
```



In [1]:
import numpy as np
from ase.cluster.wulff import wulff_construction
from ase.data import atomic_numbers, reference_states
import py3Dmol
import datetime
import os

In [2]:
os.chdir("/home/william-ackerman/Desktop/Nanocrystal_Models/NP_XYZ_Models")

### Functions:

- def validate_size(diameter_nm, max_size=60.0, min_size=1.0):
- def estimate_atom_count_fcc(metal, diameter_angstrom):
- def estimate_atom_count_bcc(metal, diameter_angstrom):
- def get_default_surface_energies_fcc(metal):
- def get_default_surface_energies_bcc(metal):
- def get_metal_color(metal):
- def print_surface_energy_summary():
- def visualize_nanoparticle(nanoparticle_data, width=800, height=600, 
                          sphere_radius=1.3, show_surface_only=False,
                           coordination_cutoff=12):
- def export_xyz(nanoparticle_data, filename=None, include_metadata=True):
- def create_wulff_nanoparticle_fcc(metal='Cu', diameter_nm=20.0, surfaces=None, energies=None, 
                              max_size=30.0, verbose=True):
- def create_wulff_nanoparticle_bcc(metal='Cr', diameter_nm=20.0, surfaces=None, energies=None, 
                                  max_size=30.0, verbose=True):
- def get_capping_limits(metal, structure='fcc'):
- def create_realistic_morphology_series(metal='Cu', diameter_nm=10.0, structure='fcc',
                                     capping_level='moderate', export_files=True):
- def size_dependent_capping_effects(metal, diameter_nm, structure='fcc'):
- def create_systematic_morphology_series(metal='Cu', diameter_nm=10.0, structure='fcc',
                                      capping_level='moderate', export_files=True, 
                                      visualize=True, show_surface_only=False):         
- def compare_morphology_energies(results):
- def create_morphology_comparison_summary(results, metal, structure):

In [3]:
def validate_size(diameter_nm, max_size=100.0, min_size=1.0):
    """
    Validate nanoparticle size and perform sanity checks.
    
    Parameters:
    -----------
    diameter_nm : float
        Target diameter in nanometers
    max_size : float
        Maximum allowed size in nm (default 30 nm)
    min_size : float
        Minimum allowed size in nm (default 1 nm)
    
    Returns:
    --------
    float : validated diameter in Angstroms
    """
    if not min_size <= diameter_nm <= max_size:
        raise ValueError(f"Size must be between {min_size} and {max_size} nm. Got {diameter_nm} nm.")
    
    diameter_angstrom = diameter_nm * 10  # Convert to Angstroms
    
    # Estimate number of atoms for warning
    target_volume = (4/3) * np.pi * (diameter_angstrom/2)**3
    # Rough atomic volume estimate (~11.8 Å³ for Cu)
    approx_atoms = int(target_volume / 12)
    
    if approx_atoms > 500000:
        print(f"Warning: Large nanoparticle (~{approx_atoms:,} atoms) may be slow to generate and visualize.")
    
    return diameter_angstrom

def estimate_atom_count_fcc(metal, diameter_angstrom):
    """
    Estimate number of atoms needed for target diameter.
    
    Parameters:
    -----------
    metal : str
        Metal symbol
    diameter_angstrom : float
        Target diameter in Angstroms
    
    Returns:
    --------
    int : estimated number of atoms
    """
    # Calculate target volume
    target_volume = (4/3) * np.pi * (diameter_angstrom/2)**3
    
    # Get atomic volume for metal
    atomic_number = atomic_numbers[metal]
    lattice_constant = reference_states[atomic_number]['a']
    atoms_per_unit_cell = 4  # for FCC
    unit_cell_volume = lattice_constant**3
    atomic_volume = unit_cell_volume / atoms_per_unit_cell
    
    # Estimate number of atoms needed
    n_atoms = int(target_volume / atomic_volume)
    
    return n_atoms

def estimate_atom_count_bcc(metal, diameter_angstrom):
    """Estimate atom count for BCC structure."""
    target_volume = (4/3) * np.pi * (diameter_angstrom/2)**3
    
    # Get atomic volume for BCC metal
    atomic_number = atomic_numbers[metal]
    lattice_constant = reference_states[atomic_number]['a']
    atoms_per_unit_cell = 2  # BCC has 2 atoms per unit cell
    unit_cell_volume = lattice_constant**3
    atomic_volume = unit_cell_volume / atoms_per_unit_cell
    
    return int(target_volume / atomic_volume)

def get_default_surface_energies_fcc(metal):
    """
    Get default surface energies for common FCC metals.
    
    Parameters:
    -----------
    metal : str
        Metal symbol
    
    Returns:
    --------
    tuple : (surfaces, energies)
        Miller indices and corresponding surface energies (J/m²)
    """
    surfaces = [(1, 0, 0), (1, 1, 1), (1, 1, 0)]
    
    # Surface energies from literature (J/m²)
    # Primary source: Vitos, L. et al. Surf. Sci. 1998, 411, 186-202
    # Secondary sources noted where different
    energy_data = {
        # Group 11 (Coinage metals)
        'Cu': [2.166, 1.952, 2.237],  # Vitos et al. 1998
        'Ag': [1.302, 1.206, 1.394],  # Vitos et al. 1998
        'Au': [1.627, 1.500, 1.694],  # Vitos et al. 1998
        
        # Group 10 (Platinum group)
        'Ni': [2.382, 2.125, 2.532],  # Vitos et al. 1998
        'Pd': [2.043, 1.796, 2.170],  # Vitos et al. 1998
        'Pt': [2.489, 2.142, 2.696],  # Vitos et al. 1998
        
        # Group 13
        'Al': [1.143, 0.855, 1.395],  # Vitos et al. 1998
        
        # Other FCC metals (from various DFT studies)
        'Pb': [0.603, 0.524, 0.735],  # Da Silva et al. Surf. Sci. 2006
        'Rh': [2.809, 2.555, 3.019],  # Calculated values
        'Ir': [3.000, 2.734, 3.234],  # Calculated values
        'Ca': [0.487, 0.421, 0.578],  # Calculated (high-T phase)
        'Sr': [0.426, 0.378, 0.501],  # Calculated (high-T phase)
        
        # Actinides (FCC at room temp or high temp)
        'Th': [1.240, 1.089, 1.367],  # Calculated values
        'Pu': [0.890, 0.756, 1.024],  # δ-Pu phase (FCC)
    }
    
    if metal not in energy_data:
        available = ', '.join(sorted(energy_data.keys()))
        raise ValueError(f"Metal '{metal}' not supported. Choose from: {available}")
    
    return surfaces, energy_data[metal]

def get_default_surface_energies_bcc(metal):
    """
    Get default surface energies for BCC metals.
    
    Parameters:
    -----------
    metal : str
        Metal symbol
    
    Returns:
    --------
    tuple : (surfaces, energies)
        Miller indices and corresponding surface energies (J/m²)
    """
    surfaces = [(1, 0, 0), (1, 1, 1), (1, 1, 0)]
    
    # BCC surface energies - NOTE: Different hierarchy than FCC!
    # (110) < (100) < (111) for BCC vs (111) < (100) < (110) for FCC
    energy_data_bcc = {
        # Group 6 (Chromium group)
        'Cr': [2.550, 3.020, 2.070],  # (100), (111), (110) - Vitos et al.
        'Mo': [3.430, 4.110, 2.900],  # (100), (111), (110)
        'W':  [4.130, 4.950, 3.490],  # (100), (111), (110)
        
        # Group 5 (Vanadium group)  
        'V':  [2.890, 3.420, 2.450],  # (100), (111), (110)
        'Nb': [2.760, 3.290, 2.340],  # (100), (111), (110) 
        'Ta': [3.140, 3.750, 2.670],  # (100), (111), (110)
        
        # Iron group (BCC phases)
        'Fe': [2.900, 3.200, 2.550],  # α-Fe (BCC), (100), (111), (110)
        
        # Alkali metals (large, soft BCC)
        'Li': [0.524, 0.612, 0.442],  # (100), (111), (110)
        'Na': [0.261, 0.305, 0.221],  # (100), (111), (110)
        'K':  [0.142, 0.166, 0.120],  # (100), (111), (110)
        
        # Alkaline earth metals (BCC at high T)
        'Ba': [0.420, 0.490, 0.355],  # (100), (111), (110)
    }
    
    if metal not in energy_data_bcc:
        available = ', '.join(sorted(energy_data_bcc.keys()))
        raise ValueError(f"BCC metal '{metal}' not supported. Choose from: {available}")
    
    return surfaces, energy_data_bcc[metal]

def get_metal_color(metal):

    """
    Get standard colors for metals based on common visualization software conventions.
    
    Parameters:
    -----------
    metal : str
        Metal symbol
    
    Returns:
    --------
    str : hex color code
    """
    # Standard colors from Ovito, VMD, and other visualization software
    metal_colors = {
        'Cu': '#B87333',  # Copper brown
        'Au': '#FFD700',  # Gold
        'Ag': '#E8E8E8',  # Silver
        'Pt': '#D0D0E0',  # Platinum (light gray-blue)
        'Pd': '#E6F3FF',  # Palladium (dark blue-green)
        'Ni': '#8F8FFF',  # Nickel (light blue)
        'Fe': '#E06633',  # Iron (orange-red)
        'Al': '#BFA6A6',  # Aluminum (light brown-gray)
    }
    
    return metal_colors.get(metal, '#808080')  # Default gray for unknown metals

def print_surface_energy_summary():
    """Print a summary of surface energy trends."""
    
    surfaces = [(1, 0, 0), (1, 1, 1), (1, 1, 0)]
    
    print("FCC Metal Surface Energies (J/m²)")
    print("=" * 50)
    print(f"{'Metal':<6} {'(100)':<8} {'(111)':<8} {'(110)':<8} {'Ratio':<8}")
    print("-" * 50)
    
    metals = ['Al', 'Ca', 'Sr', 'Ag', 'Au', 'Pb', 'Pu', 'Th', 'Cu', 'Pd', 'Ni', 'Pt', 'Rh', 'Ir']
    
    for metal in metals:
        try:
            _, energies = get_default_surface_energies_fcc(metal)
            ratio = energies[0] / energies[1]  # (100)/(111) ratio
            print(f"{metal:<6} {energies[0]:<8.3f} {energies[1]:<8.3f} {energies[2]:<8.3f} {ratio:<8.3f}")
        except ValueError:
            continue
    
    print("\nGeneral trends:")
    print("- (111) surface typically has lowest energy (most stable)")
    print("- (110) surface typically has highest energy (least stable)")
    print("- (100)/(111) ratio > 1.0 (except for some edge cases)")
    print("- Noble metals (Au, Ag, Cu) have moderate surface energies")
    print("- Platinum group metals have high surface energies")
    print("- Al has relatively low surface energies")

def visualize_nanoparticle(nanoparticle_data, width=800, height=600, 
                          sphere_radius=1.4, show_surface_only=False,
                          coordination_cutoff=12):
    """
    Visualize nanoparticle using py3Dmol with metal-appropriate colors.
    
    Parameters:
    -----------
    nanoparticle_data : dict
        Output from create_wulff_nanoparticle function
    width, height : int
        Viewer dimensions
    sphere_radius : float
        Atom sphere radius for visualization
    show_surface_only : bool
        If True, only show surface atoms (coord < coordination_cutoff)
    coordination_cutoff : int
        Coordination number cutoff for surface atoms (default 12 for FCC)
    
    Returns:
    --------
    py3Dmol viewer object
    """
    metal = nanoparticle_data['metal']
    xyz_data = nanoparticle_data['xyz_data']
    n_atoms = nanoparticle_data['n_atoms']
    
    # Get metal color
    metal_color = get_metal_color(metal)
    
    print(f"Visualizing {metal} nanoparticle with {n_atoms:,} atoms...")
    
    # Check if we need to sample for performance
    if n_atoms > 20000 and not show_surface_only:
        print(f"Warning: Large nanoparticle ({n_atoms:,} atoms).")
        print("Consider setting show_surface_only=True for better performance.")
    
    # Handle surface-only visualization
    if show_surface_only and n_atoms > 10000:
        # Calculate coordination numbers and extract surface atoms
        from ase.neighborlist import NeighborList
        atoms = nanoparticle_data['atoms']
        
        # Get lattice constant and cutoff
        atomic_number = atomic_numbers[metal]
        lattice_constant = reference_states[atomic_number]['a']
        nn_distance = lattice_constant * np.sqrt(2)/2
        cutoff = nn_distance * 1.15
        
        # Calculate coordination
        nl = NeighborList([cutoff/2]*len(atoms), self_interaction=False, bothways=True)
        nl.update(atoms)
        
        surface_atoms = []
        for i in range(len(atoms)):
            indices, offsets = nl.get_neighbors(i)
            if len(indices) < coordination_cutoff:
                surface_atoms.append(i)
        
        # Create surface-only XYZ data
        positions = atoms.get_positions()
        symbols = atoms.get_chemical_symbols()
        surface_positions = positions[surface_atoms]
        surface_symbols = [symbols[i] for i in surface_atoms]
        
        xyz_lines = [
            f"{len(surface_positions)}",
            f"{metal} Wulff Nanoparticle (surface atoms only)"
        ]
        xyz_lines.extend([f"{symbol} {x:.6f} {y:.6f} {z:.6f}" 
                         for symbol, (x, y, z) in zip(surface_symbols, surface_positions)])
        xyz_data = "\n".join(xyz_lines)
        
        print(f"Showing {len(surface_atoms):,} surface atoms (out of {n_atoms:,} total)")
    
    # Create py3Dmol viewer
    viewer = py3Dmol.view(width=width, height=height)
    viewer.addModel(xyz_data, 'xyz')
    
    # Set style with metal-appropriate color
    viewer.setStyle({'sphere': {'color': metal_color, 'radius': sphere_radius}})
    
    # Add some nice lighting and background
    viewer.setBackgroundColor('white')
    viewer.zoomTo()
    
    print(f"Visualization created with color: {metal_color}")
    return viewer

def export_xyz(nanoparticle_data, filename=None, include_metadata=True):
    """
    Export nanoparticle XYZ data to file.
    
    Parameters:
    -----------
    nanoparticle_data : dict
        Output from create_wulff_nanoparticle function
    filename : str, optional
        Output filename. If None, auto-generate based on metadata
    include_metadata : bool
        Include timestamp and parameters in filename
    
    Returns:
    --------
    str : path to written file
    """
    metal = nanoparticle_data['metal']
    diameter_nm = nanoparticle_data['diameter_nm']
    n_atoms = nanoparticle_data['n_atoms']
    xyz_data = nanoparticle_data['xyz_data']
    
    # Generate filename if not provided
    if filename is None:
        if include_metadata:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{metal.lower()}_wulff_{diameter_nm:.1f}nm_{n_atoms}atoms_{timestamp}.xyz"
        else:
            filename = f"{metal.lower()}_wulff_{diameter_nm:.1f}nm.xyz"
    
    # Ensure .xyz extension
    if not filename.endswith('.xyz'):
        filename += '.xyz'
    
    # Write file
    try:
        with open(filename, 'w') as f:
            f.write(xyz_data)
        
        file_size = os.path.getsize(filename) / (1024 * 1024)  # MB
        print(f"XYZ file written successfully:")
        print(f"  Filename: {filename}")
        print(f"  Atoms: {n_atoms:,}")
        print(f"  File size: {file_size:.2f} MB")
        
        return filename
        
    except Exception as e:
        print(f"Error writing file: {e}")
        return None
    
def create_wulff_nanoparticle_fcc(metal='Cu', diameter_nm=20.0, surfaces=None, energies=None, 
                              max_size=100.0, verbose=True):
    """
    Create a Wulff construction nanoparticle.
    
    Parameters:
    -----------
    metal : str
        Metal symbol ('Cu', 'Ag', 'Au'), default 'Cu'
    diameter_nm : float
        Target diameter in nanometers, default 20.0
    surfaces : list of tuples, optional
        Miller indices, defaults to [(1,0,0), (1,1,1), (1,1,0)]
    energies : list of floats, optional
        Surface energies, defaults to literature values
    max_size : float
        Maximum allowed size in nm, default 30.0
    verbose : bool
        Print progress information, default True
    
    Returns:
    --------
    dict : Dictionary containing:
        - 'atoms': ASE Atoms object
        - 'xyz_data': XYZ format string
        - 'metal': metal symbol
        - 'diameter_nm': actual diameter in nm
        - 'n_atoms': number of atoms
        - 'surfaces': Miller indices used
        - 'energies': surface energies used
    """
    
    # Validate inputs
    diameter_angstrom = validate_size(diameter_nm, max_size)
    
    # Get surface data
    if surfaces is None or energies is None:
        surfaces, energies = get_default_surface_energies_fcc(metal)
    
    if verbose:
        print(f"Creating {metal} nanoparticle:")
        print(f"  Target diameter: {diameter_nm} nm ({diameter_angstrom} Å)")
        print(f"  Surface energies: {dict(zip(['(100)', '(111)', '(110)'], energies))}")
    
    # Estimate atoms needed
    n_atoms = estimate_atom_count_fcc(metal, diameter_angstrom)
    
    if verbose:
        print(f"  Estimated atoms needed: {n_atoms:,}")
    
    # Create Wulff construction
    atoms = wulff_construction(metal, surfaces=surfaces, energies=energies, 
                              size=n_atoms, structure='fcc', rounding='above')
    
    # Calculate actual diameter
    positions = atoms.get_positions()
    center = atoms.get_center_of_mass()
    distances = np.sqrt(np.sum((positions - center)**2, axis=1))
    actual_diameter_angstrom = 2 * max(distances)
    actual_diameter_nm = actual_diameter_angstrom / 10
    
    # Create XYZ data string
    symbols = atoms.get_chemical_symbols()
    xyz_lines = [
        f"{len(positions)}",
        f"{metal} Wulff Nanoparticle (diameter {actual_diameter_nm:.1f} nm)"
    ]
    xyz_lines.extend([f"{symbol} {x:.6f} {y:.6f} {z:.6f}" 
                     for symbol, (x, y, z) in zip(symbols, positions)])
    xyz_data = "\n".join(xyz_lines)
    
    if verbose:
        print(f"  Actual atoms: {len(atoms):,}")
        print(f"  Actual diameter: {actual_diameter_nm:.2f} nm")
    
    return {
        'atoms': atoms,
        'xyz_data': xyz_data,
        'metal': metal,
        'diameter_nm': actual_diameter_nm,
        'n_atoms': len(atoms),
        'surfaces': surfaces,
        'energies': energies
    }

def create_wulff_nanoparticle_bcc(metal='Cr', diameter_nm=20.0, surfaces=None, energies=None, 
                                  max_size=100.0, verbose=True):
    """
    Create a Wulff construction nanoparticle for BCC metals.
    
    Key difference: Uses 'bcc' structure instead of 'fcc'
    """
    
    # Validate inputs
    diameter_angstrom = validate_size(diameter_nm, max_size)
    
    # Get surface data for BCC
    if surfaces is None or energies is None:
        surfaces, energies = get_default_surface_energies_bcc(metal)
    
    if verbose:
        print(f"Creating BCC {metal} nanoparticle:")
        print(f"  Target diameter: {diameter_nm} nm ({diameter_angstrom} Å)")
        print(f"  Surface energies: {dict(zip(['(100)', '(111)', '(110)'], energies))}")
        print(f"  Note: BCC hierarchy is (110) < (100) < (111)")
    
    # Estimate atoms needed - different for BCC structure
    n_atoms = estimate_atom_count_bcc(metal, diameter_angstrom)
    
    if verbose:
        print(f"  Estimated atoms needed: {n_atoms:,}")
    
    # Create Wulff construction with BCC structure
    atoms = wulff_construction(metal, surfaces=surfaces, energies=energies, 
                              size=n_atoms, structure='bcc', rounding='above')  # ← BCC here!
    
    # Rest is same as FCC version...
    positions = atoms.get_positions()
    center = atoms.get_center_of_mass()
    distances = np.sqrt(np.sum((positions - center)**2, axis=1))
    actual_diameter_angstrom = 2 * max(distances)
    actual_diameter_nm = actual_diameter_angstrom / 10
    
    # Create XYZ data string
    symbols = atoms.get_chemical_symbols()
    xyz_lines = [
        f"{len(positions)}",
        f"BCC {metal} Wulff Nanoparticle (diameter {actual_diameter_nm:.1f} nm)"
    ]
    xyz_lines.extend([f"{symbol} {x:.6f} {y:.6f} {z:.6f}" 
                     for symbol, (x, y, z) in zip(symbols, positions)])
    xyz_data = "\n".join(xyz_lines)
    
    if verbose:
        print(f"  Actual atoms: {len(atoms):,}")
        print(f"  Actual diameter: {actual_diameter_nm:.2f} nm")
    
    return {
        'atoms': atoms,
        'xyz_data': xyz_data,
        'metal': metal,
        'diameter_nm': actual_diameter_nm,
        'n_atoms': len(atoms),
        'surfaces': surfaces,
        'energies': energies,
        'structure': 'bcc'
    }

def get_capping_limits(metal, structure='fcc'):
    """
    Get realistic limits for surface energy modification by capping agents.
    
    Parameters:
    -----------
    metal : str
        Metal symbol
    structure : str
        Crystal structure ('fcc' or 'bcc')
        
    Returns:
    --------
    dict : Dictionary with capping limits and factors
    """
    
    # Get default surface energies
    if structure == 'fcc':
        surfaces, default_energies = get_default_surface_energies_fcc(metal)
    else:
        surfaces, default_energies = get_default_surface_energies_bcc(metal)
    
    # Capping effectiveness depends on metal properties
    metal_properties = {
        # Noble metals - harder to cap, strong metallic bonding
        'Au': {'min_factor': 0.15, 'typical_factor': 0.3, 'max_reduction': 0.85},
        'Ag': {'min_factor': 0.12, 'typical_factor': 0.25, 'max_reduction': 0.88},
        'Cu': {'min_factor': 0.10, 'typical_factor': 0.20, 'max_reduction': 0.90},
        'Pt': {'min_factor': 0.20, 'typical_factor': 0.35, 'max_reduction': 0.80},
        'Pd': {'min_factor': 0.15, 'typical_factor': 0.30, 'max_reduction': 0.85},
        
        # More reactive metals - easier to cap
        'Ni': {'min_factor': 0.08, 'typical_factor': 0.18, 'max_reduction': 0.92},
        'Al': {'min_factor': 0.05, 'typical_factor': 0.15, 'max_reduction': 0.95},
        'Fe': {'min_factor': 0.10, 'typical_factor': 0.20, 'max_reduction': 0.90},
        'Cr': {'min_factor': 0.12, 'typical_factor': 0.25, 'max_reduction': 0.88},
        'Mo': {'min_factor': 0.15, 'typical_factor': 0.30, 'max_reduction': 0.85},
        'W':  {'min_factor': 0.18, 'typical_factor': 0.35, 'max_reduction': 0.82},
        
        # Highly reactive metals - very easy to cap
        'Li': {'min_factor': 0.03, 'typical_factor': 0.10, 'max_reduction': 0.97},
        'Na': {'min_factor': 0.03, 'typical_factor': 0.10, 'max_reduction': 0.97},
        'K':  {'min_factor': 0.03, 'typical_factor': 0.10, 'max_reduction': 0.97},
    }
    
    # Default values for unlisted metals
    default_props = {'min_factor': 0.15, 'typical_factor': 0.30, 'max_reduction': 0.85}
    props = metal_properties.get(metal, default_props)
    
    # Calculate absolute minimum energies
    min_energies = [E * props['min_factor'] for E in default_energies]
    typical_min_energies = [E * props['typical_factor'] for E in default_energies]
    
    return {
        'metal': metal,
        'structure': structure,
        'default_energies': default_energies,
        'absolute_minimum': min_energies,
        'typical_minimum': typical_min_energies,
        'min_factor': props['min_factor'],
        'typical_factor': props['typical_factor'],
        'max_reduction': props['max_reduction'],
        'surfaces': surfaces
    }

def create_realistic_morphology_series(metal='Cu', diameter_nm=10.0, structure='fcc',
                                     capping_level='moderate', export_files=True):
    """
    Create morphology series with realistic surface energy modifications.
    
    Parameters:
    -----------
    metal : str
        Metal symbol
    diameter_nm : float
        Target diameter
    structure : str
        Crystal structure ('fcc' or 'bcc')
    capping_level : str
        Capping effectiveness: 'none', 'weak', 'moderate', 'strong', 'extreme'
    export_files : bool
        Export files for each morphology
    """
    
    # Get capping limits
    limits = get_capping_limits(metal, structure)
    default_energies = limits['default_energies']
    surfaces = limits['surfaces']
    
    # Define capping levels
    capping_factors = {
        'none': 1.0,        # No capping (bare metal)
        'weak': 0.8,        # 20% reduction
        'moderate': 0.6,    # 40% reduction  
        'strong': 0.4,      # 60% reduction
        'extreme': limits['typical_factor']  # Near theoretical minimum
    }
    
    base_factor = capping_factors.get(capping_level, 0.6)
    
    # Ensure we don't go below realistic minimums
    min_allowed = limits['typical_minimum']
    
    print(f"Creating realistic morphology series for {metal} ({structure.upper()})")
    print(f"Capping level: {capping_level} (factor: {base_factor})")
    print(f"Default energies: {default_energies}")
    print(f"Minimum allowed: {min_allowed}")
    
    # Create morphologies with realistic constraints
    morphologies = {}
    
    # 1. Default/bare metal
    morphologies['bare_metal'] = {
        'energies': default_energies,
        'description': 'Bare metal surface (no capping)'
    }
    
    # 2. Isotropic capping (all surfaces equally affected)
    iso_energies = [max(E * base_factor, min_E) 
                   for E, min_E in zip(default_energies, min_allowed)]
    morphologies['isotropic_capped'] = {
        'energies': iso_energies,
        'description': f'Isotropic capping ({capping_level})'
    }
    
    # 3. Selective capping scenarios
    if structure == 'fcc':
        # FCC: (100), (111), (110)
        
        # Stabilize {111} faces (common with organic ligands)
        selective_111 = [
            default_energies[0] * (base_factor + 0.2),  # {100} less affected
            max(default_energies[1] * base_factor, min_allowed[1]),  # {111} strongly capped
            default_energies[2] * (base_factor + 0.3)   # {110} least affected
        ]
        morphologies['111_selective'] = {
            'energies': selective_111,
            'description': 'Selective {111} capping (octahedral growth)'
        }
        
        # Stabilize {100} faces (common with certain surfactants)
        selective_100 = [
            max(default_energies[0] * base_factor, min_allowed[0]),  # {100} strongly capped
            default_energies[1] * (base_factor + 0.2),  # {111} less affected
            default_energies[2] * (base_factor + 0.3)   # {110} least affected
        ]
        morphologies['100_selective'] = {
            'energies': selective_100,
            'description': 'Selective {100} capping (cubic growth)'
        }
        
        # Dual capping {111} and {100} (cuboctahedral)
        dual_cap = [
            max(default_energies[0] * (base_factor + 0.1), min_allowed[0]),  # {100}
            max(default_energies[1] * (base_factor + 0.1), min_allowed[1]),  # {111}
            default_energies[2] * (base_factor + 0.4)   # {110} much less affected
        ]
        morphologies['dual_capped'] = {
            'energies': dual_cap,
            'description': 'Dual {111}/{100} capping (cuboctahedral)'
        }
        
    else:  # BCC
        # BCC: (100), (111), (110) - remember (110) is most stable
        
        # Enhance natural {110} preference
        selective_110 = [
            default_energies[0] * (base_factor + 0.2),  # {100}
            default_energies[1] * (base_factor + 0.3),  # {111} 
            max(default_energies[2] * base_factor, min_allowed[2])  # {110} strongly capped
        ]
        morphologies['110_selective'] = {
            'energies': selective_110,
            'description': 'Enhanced {110} capping (rhombic dodecahedral)'
        }
        
        # Stabilize {100} to compete with {110}
        selective_100_bcc = [
            max(default_energies[0] * base_factor, min_allowed[0]),  # {100} strongly capped
            default_energies[1] * (base_factor + 0.3),  # {111} less affected
            default_energies[2] * (base_factor + 0.1)   # {110} slightly affected
        ]
        morphologies['100_vs_110'] = {
            'energies': selective_100_bcc,
            'description': '{100} vs {110} competition'
        }
    
    # Create nanoparticles
    results = {}
    create_func = create_wulff_nanoparticle_bcc if structure == 'bcc' else create_wulff_nanoparticle_fcc
    
    for morph_name, morph_data in morphologies.items():
        print(f"\n--- Creating {morph_name} ---")
        energies = morph_data['energies']
        
        result = create_func(metal, diameter_nm, surfaces=surfaces, 
                           energies=energies, verbose=False)
        result['description'] = morph_data['description']
        result['capping_level'] = capping_level
        
        results[morph_name] = result
        
        print(f"{morph_name}: {result['n_atoms']:,} atoms")
        print(f"  Energies: {energies}")
        print(f"  Description: {morph_data['description']}")
        
        # Export if requested
        if export_files:
            filename = f"{metal.lower()}_{structure}_{morph_name}_{capping_level}_{diameter_nm:.0f}nm.xyz"
            export_xyz(result, filename, include_metadata=False)
    
    return results, limits

def size_dependent_capping_effects(metal, diameter_nm, structure='fcc'):
    """
    Model size-dependent capping effects.
    Smaller particles have higher surface area/volume ratio, more capping effect.
    """
    
    # Size effects (empirical relationships)
    # Smaller particles: more surface atoms, stronger capping effects
    if diameter_nm < 2.0:
        size_factor = 0.7   # Strong capping effect
    elif diameter_nm < 5.0:
        size_factor = 0.8   # Moderate effect
    elif diameter_nm < 10.0:
        size_factor = 0.9   # Weak effect
    else:
        size_factor = 0.95  # Minimal effect
    
    # Get base limits
    limits = get_capping_limits(metal, structure)
    
    # Adjust based on size
    adjusted_limits = limits.copy()
    adjusted_limits['size_adjusted_min'] = [E * size_factor for E in limits['typical_minimum']]
    adjusted_limits['size_factor'] = size_factor
    adjusted_limits['diameter_nm'] = diameter_nm
    
    return adjusted_limits

def create_systematic_morphology_series(metal='Cu', diameter_nm=10.0, structure='fcc',
                                      capping_level='moderate', export_files=True, 
                                      visualize=True, show_surface_only=False,max_size=100.0):
    """
    Create systematic morphology series with specific surface stabilizations.
    
    Generates 8 morphologies:
    1. Reference bare metal
    2. {100} stabilized
    3. {110} stabilized  
    4. {111} stabilized
    5. {100} and {110} stabilized
    6. {100} and {111} stabilized
    7. {110} and {111} stabilized
    8. All three stabilized
    
    Parameters:
    -----------
    metal : str
        Metal symbol
    diameter_nm : float
        Target diameter
    structure : str
        Crystal structure ('fcc' or 'bcc')
    capping_level : str
        Capping effectiveness: 'none', 'weak', 'moderate', 'strong'
    export_files : bool
        Export XYZ files
    visualize : bool
        Create visualization objects
    show_surface_only : bool
        Pass to visualize_nanoparticle() to show only surface atoms (default False)
        
    Returns:
    --------
    dict : Results for each morphology
    dict : Viewer objects (if visualize=True)
    """
    
    print(f"Creating systematic morphology series for {metal} ({structure.upper()})")
    print(f"Diameter: {diameter_nm} nm, Capping level: {capping_level}")
    if visualize:
        print(f"Visualization: Surface only = {show_surface_only}")
    print("=" * 60)
    
    # Get default energies and limits
    limits = get_capping_limits(metal, structure)
    default_energies = limits['default_energies']
    surfaces = limits['surfaces']
    min_allowed = limits['typical_minimum']
    
    # Define capping strength
    capping_factors = {
        'none': 1.0,        # No capping
        'weak': 0.7,        # 30% reduction
        'moderate': 0.5,    # 50% reduction  
        'strong': 0.3,      # 70% reduction
    }
    
    stabilization_factor = capping_factors.get(capping_level, 0.5)
    
    print(f"Default energies: {default_energies}")
    print(f"Stabilization factor: {stabilization_factor}")
    print(f"Minimum allowed: {min_allowed}")
    print()
    
    # Define the 8 morphologies
    # surfaces = [(1, 0, 0), (1, 1, 1), (1, 1, 0)]  # {100}, {111}, {110}
    morphologies = {}
    
    # 0. Reference bare metal (FIXED: Changed key name)
    energies_bare = [
        default_energies[0],  # {100} unchanged
        default_energies[1],  # {111} unchanged
        default_energies[2]   # {110} unchanged
    ]
    morphologies['bare_metal'] = {  # FIXED: Unique key name
        'energies': energies_bare,
        'description': 'Bare metal, no surfaces stabilized'
    }

    # 1. {100} stabilized only
    energies_100 = [
        max(default_energies[0] * stabilization_factor, min_allowed[0]),  # {100} stabilized
        default_energies[1],  # {111} unchanged
        default_energies[2]   # {110} unchanged
    ]
    morphologies['100_stabilized'] = {
        'energies': energies_100,
        'description': 'Only {100} faces stabilized'
    }
    
    # 2. {110} stabilized only  
    energies_110 = [
        default_energies[0],  # {100} unchanged
        default_energies[1],  # {111} unchanged
        max(default_energies[2] * stabilization_factor, min_allowed[2])   # {110} stabilized
    ]
    morphologies['110_stabilized'] = {
        'energies': energies_110,
        'description': 'Only {110} faces stabilized'
    }
    
    # 3. {111} stabilized only
    energies_111 = [
        default_energies[0],  # {100} unchanged
        max(default_energies[1] * stabilization_factor, min_allowed[1]),  # {111} stabilized
        default_energies[2]   # {110} unchanged
    ]
    morphologies['111_stabilized'] = {
        'energies': energies_111,
        'description': 'Only {111} faces stabilized'
    }
    
    # 4. {100} and {110} stabilized
    energies_100_110 = [
        max(default_energies[0] * stabilization_factor, min_allowed[0]),  # {100} stabilized
        default_energies[1],  # {111} unchanged
        max(default_energies[2] * stabilization_factor, min_allowed[2])   # {110} stabilized
    ]
    morphologies['100_110_stabilized'] = {
        'energies': energies_100_110,
        'description': '{100} and {110} faces stabilized'
    }
    
    # 5. {100} and {111} stabilized (cuboctahedral)
    energies_100_111 = [
        max(default_energies[0] * stabilization_factor, min_allowed[0]),  # {100} stabilized
        max(default_energies[1] * stabilization_factor, min_allowed[1]),  # {111} stabilized
        default_energies[2]   # {110} unchanged
    ]
    morphologies['100_111_stabilized'] = {
        'energies': energies_100_111,
        'description': '{100} and {111} faces stabilized (cuboctahedral)'
    }
    
    # 6. {110} and {111} stabilized
    energies_110_111 = [
        default_energies[0],  # {100} unchanged
        max(default_energies[1] * stabilization_factor, min_allowed[1]),  # {111} stabilized
        max(default_energies[2] * stabilization_factor, min_allowed[2])   # {110} stabilized
    ]
    morphologies['110_111_stabilized'] = {
        'energies': energies_110_111,
        'description': '{110} and {111} faces stabilized'
    }
    
    # 7. All three stabilized
    energies_all = [
        max(default_energies[0] * stabilization_factor, min_allowed[0]),  # {100} stabilized
        max(default_energies[1] * stabilization_factor, min_allowed[1]),  # {111} stabilized
        max(default_energies[2] * stabilization_factor, min_allowed[2])   # {110} stabilized
    ]
    morphologies['all_stabilized'] = {
        'energies': energies_all,
        'description': 'All {100}, {111}, and {110} faces stabilized'
    }
    
    # Create nanoparticles
    results = {}
    viewers = {}
    
    # Choose creation function based on structure
    if structure == 'fcc':
        create_func = create_wulff_nanoparticle_fcc
    else:
        create_func = create_wulff_nanoparticle_bcc
    
    print("Creating nanoparticles:")
    print("-" * 40)
    
    for i, (morph_name, morph_data) in enumerate(morphologies.items(), 1):
        print(f"{i}. Creating {morph_name}...")
        
        energies = morph_data['energies']
        
        # Create nanoparticle
        result = create_func(metal, diameter_nm, surfaces=surfaces, 
                           energies=energies, verbose=False, max_size=max_size)
        result['description'] = morph_data['description']
        result['morphology_type'] = morph_name
        result['capping_level'] = capping_level
        
        results[morph_name] = result
        
        print(f"   Atoms: {result['n_atoms']:,}")
        print(f"   Energies: {[f'{e:.3f}' for e in energies]}")
        print(f"   Description: {morph_data['description']}")
        
        # Export XYZ file
        if export_files:
            filename = f"{metal.lower()}_{structure}_{morph_name}_{capping_level}_{diameter_nm:.0f}nm.xyz"
            export_xyz(result, filename, include_metadata=False)
            print(f"   Exported: {filename}")
        
        # Create viewer with specified show_surface_only setting
        if visualize:
            viewer = visualize_nanoparticle(result, show_surface_only=show_surface_only)
            viewers[f'viewer{i}'] = viewer
            print(f"   Viewer: viewer{i} (surface_only={show_surface_only})")
        
        print()
    
    print("=" * 60)
    print("Summary:")
    print(f"Created {len(results)} morphologies for {metal} {structure.upper()}")
    
    if visualize:
        print(f"\nVisualization settings: show_surface_only = {show_surface_only}")
        print("\nViewer assignments:")
        viewer_assignments = [
            "viewer1: bare_metal",
            "viewer2: 100_stabilized",
            "viewer3: 110_stabilized", 
            "viewer4: 111_stabilized",
            "viewer5: 100_110_stabilized",
            "viewer6: 100_111_stabilized",
            "viewer7: 110_111_stabilized", 
            "viewer8: all_stabilized"
        ]
        for assignment in viewer_assignments:
            print(f"  {assignment}")
        
        print(f"\nTo view: viewer1.show(), viewer2.show(), etc.")
        
        # Add performance note
        total_atoms = sum(result['n_atoms'] for result in results.values())
        avg_atoms = total_atoms // len(results)
        if not show_surface_only and avg_atoms > 20000:
            print(f"\nNote: Large structures (~{avg_atoms:,} atoms avg). Consider show_surface_only=True for better performance.")
    
    if export_files:
        print(f"\nXYZ files exported with pattern:")
        print(f"  {metal.lower()}_{structure}_[morphology]_{capping_level}_{diameter_nm:.0f}nm.xyz")
    
    return results, viewers if visualize else {}

def compare_morphology_energies(results):
    """
    Compare surface energies across all morphologies in a nice table.
    """
    print("\nSurface Energy Comparison:")
    print("=" * 80)
    print(f"{'Morphology':<25} {'(100)':<8} {'(111)':<8} {'(110)':<8} {'Description':<30}")
    print("-" * 80)
    
    for morph_name, data in results.items():
        energies = data['energies']
        desc = data['description'][:29]  # Truncate if too long
        print(f"{morph_name:<25} {energies[0]:<8.3f} {energies[1]:<8.3f} {energies[2]:<8.3f} {desc:<30}")

def create_morphology_comparison_summary(results, metal, structure):
    """
    Create a summary file comparing all morphologies.
    """
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{metal.lower()}_{structure}_morphology_comparison_{timestamp}.txt"
    
    with open(filename, 'w') as f:
        f.write(f"Morphology Comparison Summary: {metal} {structure.upper()}\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write("Surface Energy Values (J/m²):\n")
        f.write("-" * 60 + "\n")
        f.write(f"{'Morphology':<25} {'(100)':<8} {'(111)':<8} {'(110)':<8}\n")
        f.write("-" * 60 + "\n")
        
        for morph_name, data in results.items():
            energies = data['energies']
            f.write(f"{morph_name:<25} {energies[0]:<8.3f} {energies[1]:<8.3f} {energies[2]:<8.3f}\n")
        
        f.write("\nDescriptions:\n")
        f.write("-" * 30 + "\n")
        for morph_name, data in results.items():
            f.write(f"{morph_name}: {data['description']}\n")
        
        f.write(f"\nAtom counts:\n")
        f.write("-" * 20 + "\n")
        for morph_name, data in results.items():
            f.write(f"{morph_name}: {data['n_atoms']:,} atoms\n")
    
    print(f"Comparison summary written to: {filename}")
    return filename



In [4]:
# Show capping limits for different metals
metals = ['Ag', 'Au', 'Al', 'Pd']
    
print("Realistic Capping Limits by Metal:")
print("=" * 50)
for metal in metals:
    limits = get_capping_limits(metal, 'fcc')
    print(f"\n{metal}:")
    print(f"  Default: {limits['default_energies']}")
    print(f"  Min possible: {limits['absolute_minimum']}")
    print(f"  Typical min: {limits['typical_minimum']}")
    print(f"  Max reduction: {limits['max_reduction']*100:.0f}%")
    
# Create realistic morphology series
print("\n" + "="*60)
cu_results, cu_limits = create_realistic_morphology_series(
    'Cu', diameter_nm=5.0, capping_level='moderate', export_files=False
)
    
# Show size effects
print("\n" + "="*60)
print("Size-dependent capping effects for Cu:")
for size in [2.0, 5.0, 10.0, 20.0]:
    size_limits = size_dependent_capping_effects('Cu', size)
    print(f"  {size:.0f} nm: size factor = {size_limits['size_factor']:.2f}")

Realistic Capping Limits by Metal:

Ag:
  Default: [1.302, 1.206, 1.394]
  Min possible: [0.15624, 0.14472, 0.16727999999999998]
  Typical min: [0.3255, 0.3015, 0.3485]
  Max reduction: 88%

Au:
  Default: [1.627, 1.5, 1.694]
  Min possible: [0.24405, 0.22499999999999998, 0.2541]
  Typical min: [0.4881, 0.44999999999999996, 0.5082]
  Max reduction: 85%

Al:
  Default: [1.143, 0.855, 1.395]
  Min possible: [0.057150000000000006, 0.04275, 0.06975]
  Typical min: [0.17145, 0.12825, 0.20925]
  Max reduction: 95%

Pd:
  Default: [2.043, 1.796, 2.17]
  Min possible: [0.30645, 0.2694, 0.32549999999999996]
  Typical min: [0.6129, 0.5388, 0.6509999999999999]
  Max reduction: 85%

Creating realistic morphology series for Cu (FCC)
Capping level: moderate (factor: 0.6)
Default energies: [2.166, 1.952, 2.237]
Minimum allowed: [0.43320000000000003, 0.3904, 0.4474]

--- Creating bare_metal ---
bare_metal: 6,205 atoms
  Energies: [2.166, 1.952, 2.237]
  Description: Bare metal surface (no capping)

--

## Applying the Functions

The following code are some examples for creating and visualizing various nanoparticles

In [8]:
# Create systematic morphology series
metals = ['Au','Ag','Cu','Pt','Pd']

for element in metals:
    create_systematic_morphology_series(
        metal=element, 
        diameter_nm=50.0, 
        structure='fcc',
        capping_level='moderate', 
        export_files=True, 
        visualize=False,
        show_surface_only=False 
    )





Creating systematic morphology series for Au (FCC)
Diameter: 50.0 nm, Capping level: moderate
Default energies: [1.627, 1.5, 1.694]
Stabilization factor: 0.5
Minimum allowed: [0.4881, 0.44999999999999996, 0.5082]

Creating nanoparticles:
----------------------------------------
1. Creating bare_metal...
   Atoms: 3,869,361
   Energies: ['1.627', '1.500', '1.694']
   Description: Bare metal, no surfaces stabilized
XYZ file written successfully:
  Filename: au_fcc_bare_metal_moderate_50nm.xyz
  Atoms: 3,869,361
  File size: 131.19 MB
   Exported: au_fcc_bare_metal_moderate_50nm.xyz

2. Creating 100_stabilized...
   Atoms: 3,940,299
   Energies: ['0.814', '1.500', '1.694']
   Description: Only {100} faces stabilized
XYZ file written successfully:
  Filename: au_fcc_100_stabilized_moderate_50nm.xyz
  Atoms: 3,940,299
  File size: 134.77 MB
   Exported: au_fcc_100_stabilized_moderate_50nm.xyz

3. Creating 110_stabilized...
   Atoms: 3,907,021
   Energies: ['1.627', '1.500', '0.847']
   Desc